In [18]:
import numpy as np
import pandas as pd

def run_cascade_single_population(adj_matrix, thr, seed_node_index):
    infected_nodes = np.zeros((adj_matrix.shape[0]))
    input_to_node = np.sum(adj_matrix, axis=0)
    activation_strengths = np.zeros((adj_matrix.shape[0]))  # Store activation strengths
    
    infected_nodes[seed_node_index] = 1
    activation_strengths[seed_node_index] = 1  # Initialize activation strength for seed node
    
    list_of_infected_nodes_per_iter = []
    list_of_activation_strengths_per_iter = []  # Store activation strengths per iteration
    
    list_of_infected_nodes_per_iter.append(np.where(infected_nodes == 1)[0].tolist())
    list_of_activation_strengths_per_iter.append(activation_strengths.tolist())  # Store initial activation strengths
    
    counter = 0
    
    while int(np.sum(infected_nodes)) < adj_matrix.shape[0]:
        if counter > 30:
            break
        
        indices_of_infected_nodes = np.where(infected_nodes == 1)[0]
        mask_array = np.zeros((adj_matrix.shape))
        mask_array[indices_of_infected_nodes, :] = 1
        
        infected_connections = adj_matrix.copy()
        infected_connections = infected_connections * mask_array
        infected_inputs = np.sum(infected_connections, axis=0)
        infected_nodes_indices = np.where(infected_inputs / input_to_node > thr)[0]
        
        # Update activation strengths for newly infected nodes
        activation_strengths[infected_nodes_indices] += 1
        
        list_of_infected_nodes_per_iter.append(infected_nodes_indices.tolist())
        list_of_activation_strengths_per_iter.append(activation_strengths.tolist())  # Store activation strengths
        
        infected_nodes[infected_nodes_indices] = 1
        counter += 1
        
    return list_of_infected_nodes_per_iter, list_of_activation_strengths_per_iter  # Return activation strengths

        
def find_thr(adj_matrix, starting_thr):
    visited_thresholds_per_node = [[] for _ in range(adj_matrix.shape[0])]  # Initialize empty lists

    for seed_node_index in range(adj_matrix.shape[0]):
        visited_thresholds_per_node[seed_node_index] = []
        thr = starting_thr
        min_required_nodes = adj_matrix.shape[0]  # Minimum number of nodes required to be infected

        for dummy_thr in range(0,1000):
            list_of_infected_nodes_per_iter, _ = run_cascade_single_population(adj_matrix, thr, seed_node_index)
            final_infected_nodes = list_of_infected_nodes_per_iter[-1]

            if len(final_infected_nodes) == min_required_nodes:  # Check if all nodes are infected
                thr *= 2  # Increase the threshold
                visited_thresholds_per_node[seed_node_index].append(thr)
            elif len(final_infected_nodes) < min_required_nodes and dummy_thr == 0:
                thr /= 2  # Decrease the threshold for the next iteration
                visited_thresholds_per_node[seed_node_index].append(thr)
            else:
                break  # Exit the loop if no more improvement is observed
        
    max_thresholds_per_node = [max(thresholds) if thresholds else starting_thr for thresholds in visited_thresholds_per_node]
    bottleneck_node = np.argmin(max_thresholds_per_node)  # Find the node with the lowest maximum threshold
    
    return max_thresholds_per_node[bottleneck_node]

def main():
    adj_matrix = pd.read_csv('/home/gabridele/Desktop/connectome_sub-100206.csv', header=None).to_numpy().astype(float)
    zero_rows = np.where(np.sum(adj_matrix, 0) == 0)[0].tolist()
    adj_matrix_clean = np.delete(adj_matrix, zero_rows, axis=0)
    adj_matrix_clean = np.delete(adj_matrix_clean, zero_rows, axis=1)
    
    starting_thr = 0.0015
    thr = find_thr(adj_matrix_clean, starting_thr)
    print('thr:', thr)

    seed_node_index_1 = np.random.randint(0, adj_matrix_clean.shape[0])
    seed_node_index_2 = np.random.randint(0, adj_matrix_clean.shape[0])

    print('seed_node_index_1:', seed_node_index_1)
    print('seed_node_index_2:', seed_node_index_2)

    inf_nodes_1, activation_strengths_1 = run_cascade_single_population(adj_matrix_clean, thr, seed_node_index_1)
    inf_nodes_2, activation_strengths_2 = run_cascade_single_population(adj_matrix_clean, thr, seed_node_index_2)
    
    print('inf_nodes_1:', inf_nodes_1)
    print('inf_nodes_2:', inf_nodes_2)

    print('activation_strengths_1:', activation_strengths_1)
    print('activation_strengths_2:', activation_strengths_2)

    sum_A = np.sum(activation_strengths_1[-1])
    sum_B = np.sum(activation_strengths_2[-1])

    print('strength of seed A:', sum_A)
    print('strength of seed B:', sum_B)

    if sum_A > sum_B:
        print(f"Seed node A, aka {seed_node_index_1}, wins the competition with total strength {sum_A}.")
    elif sum_A < sum_B:
        print(f"Seed node B, aka {seed_node_index_2}, wins the competition with total strength {sum_B}.")
    else:
        print("Tie")
    
if __name__ == "__main__":
    main()

thr: 0.00075
seed_node_index_1: 418
seed_node_index_2: 738
strength of seed A: 2246.0
strength of seed B: 1951.0
Seed node A, aka 418, wins the competition with total strength 2246.0.
